# Config

In [1]:
# Import dependencies
import pandas as pd
from datetime import datetime
from sleeperpy import User, Leagues
from ffpackage.viz import compareFranchises
import plotly
import plotly.express as px

In [2]:
# Settings
date_string = datetime.today().strftime('%Y%m%d')
weekStart = 3
weekEnd = 17
def_starting_pts = 18
pr_bonus = 0
kr_bonus = 0
username = 'dirtywizard'

# Extract

In [3]:
# Read database
preds = pd.read_csv('db/records_preds.csv')
preds = preds.loc[preds['date_prediction']==preds['date_prediction'].max()]
preds = preds.loc[(preds['week']>=weekStart)&(preds['week']<=weekEnd)]
mults = pd.read_csv('db/pts_multipliers.csv')
rostLims = pd.read_csv('db/roster_limits.csv')

In [4]:
def processRosters(league_id):
    # Get rosters for all league members
    rosters = pd.DataFrame(Leagues.get_rosters(league_id))
    # Explode the player columns to find how they are placed on the roster
    result = pd.DataFrame(columns=['owner_id', 'player_id'])
    for col in ['taxi', 'starters', 'players']:
        df = rosters[['owner_id', col]].explode(col)
        df = df.rename(columns={col:'player_id'})
        df[col] = True
        result = result.merge(df, how='outer', on=['owner_id', 'player_id'])
    return result

In [5]:
def procesOwners(league_id):
    owners = pd.DataFrame(Leagues.get_users(league_id))
    list = []
    for i in range(len(owners)):
        if 'team_name' in owners['metadata'][i]:
            list.append(owners['metadata'][i]['team_name'])
        else:
            list.append(owners['display_name'][i])
    owners['franchise_name'] = list
    owners = owners[['user_id', 'franchise_name', 'display_name']]
    return owners

In [6]:
# Get User's Sleeper ID
user_id = User.get_user(username)['user_id']
# Get the User's Sleeper league ID
league_id = Leagues.get_all_leagues(user_id, 'nfl', 2023)[0]['league_id']
# Get all players on rosters
rosters = processRosters(league_id)
# Get info on all owners
owners = procesOwners(league_id)
# Get league info
user_league = Leagues.get_league(league_id)

# Get roster positions 
rosterList = user_league['roster_positions']
# Sort rosterList
sort_order = ['QB', 'RB', 'WR', 'TE', 'K', 'DEF', 'FLEX', 'SUPER_FLEX', 'BN']
rosterList.sort(key=sort_order.index)

# Get number of franchises
number_of_franchises = user_league['total_rosters']

# Transform

In [7]:
# Customize predictions for league
# Calculate projected points according to that league's scoring settings
# Multiply by points multiplier
for col in mults.columns[1:]:
    preds[col] = preds[col] * mults.loc[mults['username']==username, col][0]
# Add up point columns
preds['pts_proj'] = preds[mults.columns[1:]].sum(axis=1)
# Add extras for defense starting values
preds.loc[preds['Yds Allowed'].notna(), 'pts_proj'] = preds.loc[preds['Yds Allowed'].notna(), 'pts_proj'] + def_starting_pts
# Add extras for Punt/kick returners
preds.loc[preds['PR']==True, 'pts_proj'] = preds.loc[preds['PR']==True, 'pts_proj'] + (pr_bonus)
preds.loc[preds['KR']==True, 'pts_proj'] = preds.loc[preds['KR']==True, 'pts_proj'] + (kr_bonus)

# Merge in point projections with franchise data
full = preds \
    .merge(rosters, how='left', on='player_id') \
    .merge(owners, how='left', left_on='owner_id', right_on='user_id')

In [16]:
full.head()

,player_id,full_name,birth_date,age,weight,height,team,position,fantasy_positions,depth_chart_order,...,KR,date_prediction,pts_proj,owner_id,taxi,starters,players,user_id,franchise_name,display_name
0,4234,Noah Brown,1996-01-06,27.0,215.0,74,HOU,WR,['WR'],NaN,...,NaN,20230918,2.007,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4234,Noah Brown,1996-01-06,27.0,215.0,74,HOU,WR,['WR'],NaN,...,NaN,20230918,2.425,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4234,Noah Brown,1996-01-06,27.0,215.0,74,HOU,WR,['WR'],NaN,...,NaN,20230918,2.425,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4234,Noah Brown,1996-01-06,27.0,215.0,74,HOU,WR,['WR'],NaN,...,NaN,20230918,2.336,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4234,Noah Brown,1996-01-06,27.0,215.0,74,HOU,WR,['WR'],NaN,...,NaN,20230918,2.007,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Create value array

In [36]:
# Initialize array
array = full[['player_id', 'week', 'position', 'pts_proj', 'franchise_name']]
# Select only rostered players
array = array.loc[array['franchise_name'].notna()]
array = array.loc[array['franchise_name']=='Croccity Body Snatchers']
array = array.loc[array['week']==6].reset_index(drop=True)
# Roster Dictionary
rosterDict = {
    'QB':['QB'], 
    'RB':['RB'],
    'WR':['WR'],
    'TE':['TE'],
    'K':['PK'],
    'DEF':['DEF'],
    'FLEX':['RB', 'WR', 'TE'],
    'SUPER_FLEX':['QB', 'RB', 'WR', 'TE'],
}
# Create a column for each possible roster spot
counter = 1
for spot in rosterList:
    if spot != "BN":
        # Select players who could have value in that role
        array.loc[array['position'].isin(rosterDict[spot]), f'{counter}_{spot}'] = array.loc[array['position'].isin(rosterDict[spot]), 'pts_proj']
        counter += 1
array = array.fillna(0)
array = array.drop(columns=['player_id', 'week','position','pts_proj','franchise_name'])

array

,1_QB,2_RB,3_RB,4_WR,5_WR,6_WR,7_TE,8_TE,9_K,10_K,11_DEF,12_DEF,13_FLEX,14_FLEX,15_SUPER_FLEX
0,10.05,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.00,0.0000,0.0000,0.000,0.000,10.050
1,0.00,14.850,14.850,0.000,0.000,0.000,0.000,0.000,0.00,0.00,0.0000,0.0000,14.850,14.850,14.850
2,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.00,-0.5721,-0.5721,0.000,0.000,0.000
3,0.00,0.000,0.000,0.000,0.000,0.000,4.500,4.500,0.00,0.00,0.0000,0.0000,4.500,4.500,4.500
4,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.00,5.5870,5.5870,0.000,0.000,0.000
5,0.00,6.310,6.310,0.000,0.000,0.000,0.000,0.000,0.00,0.00,0.0000,0.0000,6.310,6.310,6.310
6,0.00,15.895,15.895,0.000,0.000,0.000,0.000,0.000,0.00,0.00,0.0000,0.0000,15.895,15.895,15.895
7,0.00,12.740,12.740,0.000,0.000,0.000,0.000,0.000,0.00,0.00,0.0000,0.0000,12.740,12.740,12.740
8,0.00,0.000,0.000,3.795,3.795,3.795,0.000,0.000,0.00,0.00,0.0000,0.0000,3.795,3.795,3.795
9,13.29,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.00,0.0000,0.0000,0.000,0.000,13.290


# Munkres

In [38]:
from munkres import Munkres, print_matrix
import sys
matrix = array.values
cost_matrix = []
for row in matrix:
    cost_row = []
    for col in row:
        # Subtract from maximum possible points to find cost matrix
        cost_row += [array.max().max() - col]
    cost_matrix += [cost_row]

m = Munkres()
indexes = m.compute(cost_matrix)
print_matrix(matrix, msg='Highest profit through this matrix:')
total = 0
for row, column in indexes:
    value = matrix[row][column]
    total += value
    print(f'({row}, {column}) -> {value}')

print(f'total profit={total}')

Highest profit through this matrix:
[              10.05,                 0.0,                 0.0,                 0.0,                 0.0,                 0.0,                 0.0,                 0.0,                 0.0,                 0.0,                 0.0,                 0.0,                 0.0,                 0.0,               10.05]
[                0.0,  14.850000000000001,  14.850000000000001,                 0.0,                 0.0,                 0.0,                 0.0,                 0.0,                 0.0,                 0.0,                 0.0,                 0.0,  14.850000000000001,  14.850000000000001,  14.850000000000001]
[                0.0,                 0.0,                 0.0,                 0.0,                 0.0,                 0.0,                 0.0,                 0.0,                 0.0,                 0.0, -0.5721000000000025, -0.5721000000000025,                 0.0,                 0.0,                 0.0]
[               